In [1]:
import sys
import os

# find and append the root directory
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))  # or ".." if in notebooks/
sys.path.append(project_root)

import numpy as np
import pandas as pd
import itertools
from tqdm import tqdm
import time

from src.problems.benchmarks import (
    rosenbrock, rastrigin, ackley,
    grad_rosenbrock, grad_rastrigin, grad_ackley
)
from src.optimizers.sa import sa_continuous
from src.optimizers.gd import gradient_descent
from src.optimizers.hybrid import sa_gd_hybrid
from src.utils.utils_experiments import bootstrap_experiment_benchmarks

# Paths
results_dir = os.path.join(project_root, "results", "gridsearch")
os.makedirs(results_dir, exist_ok=True)

In [2]:
benchmarks = {
    "rosenbrock": (rosenbrock, grad_rosenbrock),
    "rastrigin": (rastrigin, grad_rastrigin),
    "ackley": (ackley, grad_ackley)
}

In [3]:
# hyperparameter grids for gridsearch
sa_grid = {
    'T0': [10, 50, 100],
    'alpha': [0.9, 0.95, 0.99, 0.995, 0.999],
    'step_size': [0.005, 0.01, 0.05, 0.1, 0.3, 0.5]
}

gd_grid = {
    'lr': [0.0001, 0.001, 0.01, 0.05, 0.1, 0.3, 0.5]
}

In [4]:
# SA gridsearch helpers
def run_one_sa(f, benchmark_name, T0, alpha, step_size, num_runs=25):
    x_star = np.array([1.0, 1.0]) if benchmark_name == "rosenbrock" else np.array([0.0, 0.0])
    result = bootstrap_experiment_benchmarks(
        sa_continuous,
        runs=num_runs,
        f=f,
        dim=2,
        name=benchmark_name,
        T0=T0, alpha=alpha, step_size=step_size,
        tol=1e-6, max_iter=20000, f_star=0.0, x_star=x_star
    )
    stats = result["stats"]
    return {
        "T0": T0, "alpha": alpha, "step_size": step_size,
        "rmse": stats["rmse"], "rmed": stats["rmed"], 
        "best": stats["best"], "worst": stats["worst"], "std": stats["std"],
        "epsilon": stats["epsilon"], "near_optimal_count": stats["near_optimal_count"],
        "mean": stats["mean"], "mse": stats["mse"], 
        "med": stats["med"], "mean_runtime": stats["mean_runtime_sec"]
    }


def grid_search_sa(f, grid, num_runs=25, benchmark_name=""):
    combos = list(itertools.product(grid["T0"], grid["alpha"], grid["step_size"]))
    total = len(combos)
    results = []

    for idx, (T0, alpha, step_size) in enumerate(combos, 1):
        print(f"[SA | {benchmark_name}] Combo {idx}/{total} — Running...")
        res = run_one_sa(f, benchmark_name, T0, alpha, step_size, num_runs)
        results.append(res)

    return pd.DataFrame(results)


In [5]:
# GD grid search helpers
def run_one_gd(f, grad, benchmark_name, lr, num_runs=25):
    x_star = np.array([1.0, 1.0]) if benchmark_name == "rosenbrock" else np.array([0.0, 0.0])
    result = bootstrap_experiment_benchmarks(
        gradient_descent,
        runs=num_runs,
        f=f,
        grad_f=grad,
        dim=2,
        name=benchmark_name,
        lr=lr, tol=1e-6, max_iter=20000, f_star=0.0, x_star=x_star
    )
    stats = result["stats"]
    return {
        "lr": lr,
        "rmse": stats["rmse"], "rmed": stats["rmed"], 
        "best": stats["best"], "worst": stats["worst"], "std": stats["std"],
        "epsilon": stats["epsilon"], "near_optimal_count": stats["near_optimal_count"],
        "mean": stats["mean"], "mse": stats["mse"], 
        "med": stats["med"], "mean_runtime": stats["mean_runtime_sec"]
    }

def grid_search_gd(f, grad, grid, num_runs=25, benchmark_name=""):
    combos = list(grid["lr"])
    total = len(combos)
    results = []

    for idx, lr in enumerate(combos, 1):
        print(f"[GD | {benchmark_name}] Combo {idx}/{total} — Running...")
        res = run_one_gd(f, grad, benchmark_name, lr, num_runs)
        results.append(res)

    return pd.DataFrame(results)


# Gridsearch standalone SA and GD

In [6]:
# # run the grid search and save results
# for name, (f, grad) in benchmarks.items():
#     print(f"\nBenchmark: {name}")

#     print("Running GD grid search")
#     df_gd = grid_search_gd(f, grad, gd_grid, num_runs=25, benchmark_name=name)
#     df_gd.to_csv(os.path.join(results_dir, f"gridsearch_gd_{name}_new.csv"), index=False)

#     print("Running SA grid search")
#     df_sa = grid_search_sa(f, sa_grid, num_runs=25, benchmark_name=name)
#     df_sa.to_csv(os.path.join(results_dir, f"gridsearch_sa_{name}_new.csv"), index=False)

# Gridsearch hybrid SA-GD

In [13]:
hybrid_grid = {
    'lr': [0.0001, 0.001, 0.01, 0.05,],
    'sigma': [0.001, 0.01, 0.1],
    'T0': [0.01, 0.1, 1.0]
}

# hybrid_grid = {
#     'lr': [0.0001, 0.001],
#     'sigma': [0.1, 0.5],
#     'T0': [0.01]
# }

ascent_method = "ascent"

In [14]:
def run_one_hybrid(f, grad, benchmark_name, lr, sigma, T0, num_runs=25):
    x_star = np.array([1.0, 1.0]) if benchmark_name == "rosenbrock" else np.array([0.0, 0.0])
    result = bootstrap_experiment_benchmarks(
        algorithm_function=sa_gd_hybrid,
        runs=num_runs,
        f=f,
        grad_f=grad,
        dim=2,
        name=benchmark_name,
        lr=lr, sigma=sigma, T0=T0,
        tol=1e-6, max_iter=20000,
        f_star=0.0, x_star=x_star, ascent_method = ascent_method
    )
    stats = result["stats"]
    return {
        "lr": lr, "sigma": sigma, "T0": T0,
        "ascent_method" : ascent_method,
        "rmse": stats["rmse"], "rmed": stats["rmed"], 
        "best": stats["best"], "worst": stats["worst"], "std": stats["std"],
        "epsilon": stats["epsilon"], "near_optimal_count": stats["near_optimal_count"],
        "mean": stats["mean"], "mse": stats["mse"], 
        "med": stats["med"], "mean_runtime": stats["mean_runtime_sec"]
    }


In [15]:
def grid_search_hybrid(f, grad, grid, num_runs=25, benchmark_name=""):
    combos = list(itertools.product(grid["lr"], grid["sigma"], grid["T0"]))
    total = len(combos)
    results = []

    for idx, (lr, sigma, T0) in enumerate(combos, 1):
        print(f"[HYBRID | {benchmark_name}] Combo {idx}/{total} — Running...")
        res = run_one_hybrid(f, grad, benchmark_name, lr, sigma, T0, num_runs)
        results.append(res)

    return pd.DataFrame(results)


In [16]:
# run the grid search and save results
for name, (f, grad) in benchmarks.items():
    print(f"\nBenchmark: {name}")

    print("Running Hybrid SA-GD grid search")
    df_hybrid = grid_search_hybrid(f, grad, hybrid_grid, num_runs=2, benchmark_name=name)
    df_hybrid.to_csv(os.path.join(results_dir, f"gridsearch_hybrid-{ascent_method}_{name}_new.csv"), index=False)


Benchmark: rosenbrock
Running Hybrid SA-GD grid search
[HYBRID | rosenbrock] Combo 1/36 — Running...
[Benchmark] Run 1/2


Clip range : (-2, 2)
[Benchmark] Run 2/2
Clip range : (-2, 2)
[HYBRID | rosenbrock] Combo 2/36 — Running...
[Benchmark] Run 1/2
Clip range : (-2, 2)
[Benchmark] Run 2/2
Clip range : (-2, 2)
[HYBRID | rosenbrock] Combo 3/36 — Running...
[Benchmark] Run 1/2
Clip range : (-2, 2)
[Benchmark] Run 2/2
Clip range : (-2, 2)
[HYBRID | rosenbrock] Combo 4/36 — Running...
[Benchmark] Run 1/2
Clip range : (-2, 2)
[Benchmark] Run 2/2
Clip range : (-2, 2)
[HYBRID | rosenbrock] Combo 5/36 — Running...
[Benchmark] Run 1/2
Clip range : (-2, 2)
[Benchmark] Run 2/2
Clip range : (-2, 2)
[HYBRID | rosenbrock] Combo 6/36 — Running...
[Benchmark] Run 1/2
Clip range : (-2, 2)
[Benchmark] Run 2/2
Clip range : (-2, 2)
[HYBRID | rosenbrock] Combo 7/36 — Running...
[Benchmark] Run 1/2
Clip range : (-2, 2)
[Benchmark] Run 2/2
Clip range : (-2, 2)
[HYBRID | rosenbrock] Combo 8/36 — Running...
[Benchmark] Run 1/2
Clip range : (-2, 2)
[Benchmark] Run 2/2
Clip range : (-2, 2)
[HYBRID | rosenbrock] Combo 9/36 — Running